## Generación geojson de Medellín
### es un archivo con las coordenadas de la ciudad de medellin

In [ ]:
import folium

# --- 1. Definición de las coordenadas del rectángulo ---
# Las coordenadas se dan en el formato: 
# Longitud_SW, Latitud_SW, Longitud_NE, Latitud_NE

# Coordenadas proporcionadas: -75.5966, 6.2350, -75.5766, 6.2550
lon_sw = -75.5966
lat_sw = 6.2350
lon_ne = -75.5766
lat_ne = 6.2550

# Límites para el rectángulo (Folium espera [ [lat_sw, lon_sw], [lat_ne, lon_ne] ] )
limites_rectangulo = [
    [lat_sw, lon_sw],  # Esquina inferior izquierda (SurOeste)
    [lat_ne, lon_ne]   # Esquina superior derecha (NorEste)
]

# --- 2. Cálculo del centro del mapa para un enfoque inicial ---
# El centro del mapa será el promedio de las latitudes y longitudes.
centro_lat = (lat_sw + lat_ne) / 2
centro_lon = (lon_sw + lon_ne) / 2

# --- 3. Creación del mapa base con Folium ---
# Creamos un mapa centrado en el medio del área y con un nivel de zoom adecuado.
m = folium.Map(
    location=[centro_lat, centro_lon], 
    zoom_start=14,  # Un buen nivel de zoom para ver el área
    tiles="OpenStreetMap"
)

# --- 4. Dibujar el rectángulo en el mapa ---
# Usamos folium.Rectangle para dibujar la forma con los límites definidos.
folium.Rectangle(
    bounds=limites_rectangulo,
    color='#ff7800',         # Color del borde (naranja)
    weight=3,                # Grosor del borde
    fill=True,               # Rellenar el rectángulo
    fill_color='#ffff00',    # Color de relleno (amarillo)
    fill_opacity=0.2,        # Transparencia del relleno
    popup='Área de Interés'  # Texto que aparece al hacer clic
).add_to(m)

# --- 5. Guardar el mapa en un archivo HTMLimport folium
from folium.plugins import Draw

# --- 1. Cálculo del centro del mapa (para empezar en Medellín) ---
# Usamos las coordenadas que ya tenías para centrar el mapa
lat_sw = 6.2350
lon_sw = -75.5966
lat_ne = 6.2550
lon_ne = -75.5766

centro_lat = (lat_sw + lat_ne) / 2
centro_lon = (lon_sw + lon_ne) / 2

# --- 2. Creación del mapa base con Folium ---
m = folium.Map(
    location=[centro_lat, centro_lon], 
    zoom_start=14,
    tiles="OpenStreetMap"
)

# --- 3. Añadir el Plugin de Dibujo ---
# Añadimos el control de dibujo al mapa
draw_control = Draw(
    export=True,  # ¡Esto añade el botón de "Descargar" ⬇️!
    filename='medellin.geojson', # Nombre del archivo que se descargará
    draw_options={
        'polygon': {'allowIntersection': False}, # Opciones para dibujar polígonos
        'rectangle': True, # Habilitar la herramienta de rectángulo
        'polyline': False, # Deshabilitar línea
        'circle': False, # Deshabilitar círculo
        'marker': False, # Deshabilitar marcador
        'circlemarker': False # Deshabilitar marcador de círculo
    }
)
draw_control.add_to(m)

# --- 4. Guardar el mapa en un archivo HTML ---
nombre_archivo = 'mapa_interactivo_dibujo.html'
m.save(nombre_archivo)

print(f"✅ ¡Mapa interactivo generado! Abre '{nombre_archivo}' en tu navegador.")
nombre_archivo = 'mapa_rectangulo.html'
m.save(nombre_archivo)

print(f"✅ ¡Mapa generado con éxito! Abre el archivo '{nombre_archivo}' en tu navegador.")

## Segmentación de Geojson de Medellín en 120 recuadros

### se genera un archivo txt con las coordenadas de los 120 sectores de la ciudad

In [15]:
import geopandas as gpd
from shapely.geometry import Polygon
import folium
import numpy as np
import json

# --- 1. CONFIGURACIÓN DEL SCRIPT ---
FILE_GEOJSON = 'medellin.geojson' 

# ¡Define el tamaño de tu celda en GRADOS!
# 1 grado ~= 111 km. (0.05 grados ~= 5.5 km)
cell_height_deg = 0.015 # (n) Alto de la celda en grados
cell_width_deg = 0.015  # (m) Ancho de la celda en grados

# --- 2. CARGAR EL ÁREA (YA ESTÁ EN GRADOS) ---
print(f"Cargando el archivo: {FILE_GEOJSON}")
try:
    # Geopandas carga por defecto en EPSG:4326 (grados)
    ciudad_gdf = gpd.read_file(FILE_GEOJSON)
except Exception as e:
    print(f"ERROR: No se pudo leer '{FILE_GEOJSON}'.")
    print(f"Detalle: {e}")
    exit()

# ¡Ya no necesitamos proyectar a metros!

# --- 3. CREAR LA CUADRÍCULA (GRID) EN GRADOS ---
# Obtener los límites totales de la ciudad (en grados)
min_x, min_y, max_x, max_y = ciudad_gdf.total_bounds # (min_lon, min_lat, max_lon, max_lat)

# Crear los "pasos" para la cuadrícula usando el tamaño en grados
grid_cols = list(np.arange(min_x, max_x + cell_width_deg, cell_width_deg))
grid_rows = list(np.arange(min_y, max_y + cell_height_deg, cell_height_deg))

M_columnas = len(grid_cols) - 1
N_filas = len(grid_rows) - 1

print(f"\n--- Dimensiones de la Cuadrícula ---")
print(f"Cada celda mide {cell_width_deg}° (ancho, m) x {cell_height_deg}° (alto, n).")
print(f"Total de celdas: {N_filas * M_columnas} (M={M_columnas}, N={N_filas}).")

polygons_list = []
for x in grid_cols[:-1]:
    for y in grid_rows[:-1]:
        polygons_list.append(Polygon([
            (x, y),
            (x + cell_width_deg, y),
            (x + cell_width_deg, y + cell_height_deg),
            (x, y + cell_height_deg)
        ]))

# Creamos el GeoDataFrame. Su CRS nativo es EPSG:4326
grid_gdf = gpd.GeoDataFrame({'geometry': polygons_list}, crs="EPSG:4326")


# --- 4. CORTAR LA CUADRÍCULA CON EL ÁREA DE LA CIUDAD ---
print("\nCortando la cuadrícula con la forma de la ciudad...")
# El clip se hace directamente en grados.
grid_cortado = gpd.clip(grid_gdf, ciudad_gdf)

# --- 5. VISUALIZAR EN UN MAPA INTERACTIVO ---
print("Generando mapa interactivo 'mapa_con_grid.html'...")
# ¡No se necesita reproyección! Los datos ya están listos para Folium
centro = ciudad_gdf.geometry.centroid.iloc[0]
mapa = folium.Map(location=[centro.y, centro.x], zoom_start=11)

folium.GeoJson(
    ciudad_gdf,
    style_function=lambda x: {'color': 'blue', 'weight': 3, 'fillOpacity': 0.1},
    name="Límite de la Ciudad"
).add_to(mapa)

folium.GeoJson(
    grid_cortado,
    style_function=lambda x: {'color': 'red', 'weight': 1, 'fillOpacity': 0.4},
    name="Cuadrícula"
).add_to(mapa)

folium.LayerControl().add_to(mapa)
mapa.save('mapa_con_grid.html')
print("¡Mapa generado!")

# --- 6. EXTRAER Y GUARDAR LOS BBOXES ---
print("\n--- Extrayendo Bounding Boxes ---")

lista_de_bboxes = []
# Iteramos sobre la geometría de nuestra cuadrícula cortada
for geometria in grid_cortado.geometry:
    # Obtenemos los límites (bounds) de cada celda.
    # Como la forma de entrada es un rectángulo y la cuadrícula es rectangular,
    # el 'clip' debería dar como resultado rectángulos.
    bounds = geometria.bounds # Devuelve (min_lon, min_lat, max_lon, max_lat)
    
    # Formateamos el string
    bbox_str = f"{bounds[0]},{bounds[1]},{bounds[2]},{bounds[3]}"
    lista_de_bboxes.append(bbox_str)

# --- Guardar la lista en un archivo TXT ---
output_file = "lista_bboxes.txt"
with open(output_file, 'w') as f:
    for bbox in lista_de_bboxes:
        f.write(f"{bbox}\n")

print(f"¡Éxito! Se extrajeron {len(lista_de_bboxes)} BBOXes.")
print(f"Fueron guardados en el archivo '{output_file}'.")

print("\n¡Proceso completado!")

Cargando el archivo: medellin.geojson

--- Dimensiones de la Cuadrícula ---
Cada celda mide 0.015° (ancho, m) x 0.015° (alto, n).
Total de celdas: 120 (M=8, N=15).

Cortando la cuadrícula con la forma de la ciudad...
Generando mapa interactivo 'mapa_con_grid.html'...
¡Mapa generado!

--- Extrayendo Bounding Boxes ---
¡Éxito! Se extrajeron 120 BBOXes.
Fueron guardados en el archivo 'lista_bboxes.txt'.

¡Proceso completado!


/tmp/ipykernel_67490/4007167768.py:64: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centro = ciudad_gdf.geometry.centroid.iloc[0]


### codigo para generar HTML con el mapa y los 120 recuadros

In [16]:
import folium
from folium.features import DivIcon
import os

# --- 1. Cargar el archivo de BBOXes ---
BBOX_FILE = 'lista_bboxes.txt'

print(f"Cargando bboxes desde '{BBOX_FILE}'...")
try:
    with open(BBOX_FILE, 'r') as f:
        # Leemos todas las líneas y quitamos espacios en blanco
        bbox_strings = [line.strip() for line in f.readlines() if line.strip()]
except FileNotFoundError:
    print(f"¡Error! No se pudo encontrar el archivo '{BBOX_FILE}'.")
    print("Asegúrate de haber ejecutado el script anterior primero.")
    # Si estás en un notebook, puedes usar 'pass' o 'raise'
    # En un script, usarías exit()
    pass 

if not bbox_strings:
    print("El archivo de BBOXes está vacío.")
else:
    # --- 2. Procesar y Ordenar los BBOXes ---
    parsed_bboxes = []
    for bbox_str in bbox_strings:
        try:
            # Convertimos el string "min_lon,min_lat,max_lon,max_lat" en números
            parts = [float(f) for f in bbox_str.split(',')]
            min_lon, min_lat, max_lon, max_lat = parts
            parsed_bboxes.append({
                "bbox_str": bbox_str,
                "min_lon": min_lon,
                "min_lat": min_lat,
                "max_lon": max_lon,
                "max_lat": max_lat,
            })
        except ValueError:
            print(f"Advertencia: Ignorando línea malformada: {bbox_str}")

    # --- ¡El Paso Clave de Ordenamiento! ---
    # Ordenamos de arriba-abajo (usando la latitud MÁXIMA en orden descendente)
    # y de izquierda-derecha (usando la longitud MÍNIMA en orden ascendente)
    sorted_bboxes = sorted(
        parsed_bboxes, 
        key=lambda item: (-item['max_lat'], item['min_lon'])
    )
    
    print(f"Se procesaron y ordenaron {len(sorted_bboxes)} BBOXes.")

    # --- 3. Crear el Mapa ---
    if not sorted_bboxes:
        print("No hay BBOXes válidos para mapear.")
    else:
        # Usamos el primer BBOX de la lista ordenada para centrar el mapa
        first_bbox = sorted_bboxes[0]
        centro_lat = (first_bbox['min_lat'] + first_bbox['max_lat']) / 2
        centro_lon = (first_bbox['min_lon'] + first_bbox['max_lon']) / 2

        mapa = folium.Map(location=[centro_lat, centro_lon], zoom_start=12)

        # --- 4. Añadir BBOXes y Etiquetas al Mapa ---
        print("Añadiendo BBOXes y etiquetas al mapa...")
        
        for i, item in enumerate(sorted_bboxes):
            
            # Definir los límites para el rectángulo de Folium: [[min_lat, min_lon], [max_lat, max_lon]]
            bounds = [[item['min_lat'], item['min_lon']], [item['max_lat'], item['max_lon']]]
            
            # Dibujar el rectángulo
            folium.Rectangle(
                bounds=bounds,
                color='blue',
                weight=1,
                fill=True,
                fill_color='blue',
                fill_opacity=0.1,
                tooltip=f"ID (Índice): {i}\n{item['bbox_str']}"
            ).add_to(mapa)
            
            # Calcular el centro para poner la etiqueta numérica
            center_lat = (item['min_lat'] + item['max_lat']) / 2
            center_lon = (item['min_lon'] + item['max_lon']) / 2
            
            # Añadir la etiqueta numérica (índice 'i')
            folium.Marker(
                location=[center_lat, center_lon],
                icon=DivIcon(
                    icon_size=(150,36),
                    icon_anchor=(7,10),
                    # Estilos CSS para crear un número visible
                    html=f'<div style="font-size: 9pt; font-weight: bold; color: black; background-color: rgba(255,255,255,0.7); border-radius: 5px; padding: 0px 3px;">{i}</div>'
                )
            ).add_to(mapa)

        # --- 5. Guardar el Mapa ---
        output_map_file = 'mapa_enumerado_bboxes.html'
        mapa.save(output_map_file)
        print(f"¡Mapa generado con éxito! Abre '{output_map_file}' para verlo.")

Cargando bboxes desde 'lista_bboxes.txt'...
Se procesaron y ordenaron 120 BBOXes.
Añadiendo BBOXes y etiquetas al mapa...
¡Mapa generado con éxito! Abre 'mapa_enumerado_bboxes.html' para verlo.


## Descarga de informacion de imagenes a usar
### se genera un archivo con la informacion y enlaces de descarga de todas las imagenes de Medellín entregadas por la página web Mapillary. Se revisa que no hayan duplicados

In [ ]:
import requests
import os
import time
from dotenv import load_dotenv
import sys
import json

# --- 1. CONFIGURACIÓN E INICIO ---
print("--- Iniciando Script de REPORTE y GENERACIÓN DE JSON ---")
load_dotenv()
ACCESS_TOKEN = os.getenv("MAPILLARY_ACCESS_TOKEN")

if not ACCESS_TOKEN:
    print("Error: No se encontró MAPILLARY_ACCESS_TOKEN.")
    sys.exit()

BBOX_FILE = 'lista_bboxes.txt'
search_url = "https://graph.mapillary.com/images"
OUTPUT_JSONL_FILE = "descarga_lista.jsonl"
# --- ¡NUEVO! Archivo de log para BBOXes completados ---
COMPLETED_LOG_FILE = "completed_bboxes.log"

# --- 2. LEER BBOXES Y CARGAR PROGRESO ANTERIOR ---
try:
    with open(BBOX_FILE, 'r') as f:
        bbox_list = [line.strip() for line in f.readlines() if line.strip()]
except FileNotFoundError:
    print(f"¡Error! No se pudo encontrar el archivo '{BBOX_FILE}'.")
    sys.exit()

print(f"Se analizarán {len(bbox_list)} BBOXes.")

# --- Cargar IDs existentes para resumir (evitar duplicados) ---
ids_globales = set()
if os.path.exists(OUTPUT_JSONL_FILE):
    print(f"Cargando IDs de imágenes desde '{OUTPUT_JSONL_FILE}'...")
    try:
        with open(OUTPUT_JSONL_FILE, 'r') as f:
            for line in f:
                if line.strip(): 
                    data = json.loads(line)
                    ids_globales.add(data['id'])
        print(f"Se cargaron {len(ids_globales)} IDs únicos ya procesados.")
    except Exception as e:
        print(f"Error al leer {OUTPUT_JSONL_FILE}, empezando de cero. Error: {e}")
        ids_globales = set()

# --- ¡NUEVO! Cargar BBOXes que ya se completaron ---
completed_bboxes_set = set()
if os.path.exists(COMPLETED_LOG_FILE):
    print(f"Cargando BBOXes completados desde '{COMPLETED_LOG_FILE}'...")
    with open(COMPLETED_LOG_FILE, 'r') as f:
        for line in f:
            if line.strip():
                try:
                    completed_bboxes_set.add(int(line.strip()))
                except ValueError:
                    pass # Ignorar líneas malformadas
    print(f"Se omitirán {len(completed_bboxes_set)} BBOXes ya completados.")

# --- 3. FASE DE REPORTE Y CAPTURA (BUCLE PRINCIPAL) ---
total_duplicados_globales = 0

# Abrimos ambos archivos de salida en modo 'append' (añadir)
try:
    with open(OUTPUT_JSONL_FILE, 'a') as f_jsonl, open(COMPLETED_LOG_FILE, 'a') as f_log:
        
        # --- BUCLE EXTERNO: Itera sobre cada BBOX ---
        for i, bbox_str in enumerate(bbox_list):
            
            bbox_index = i + 1
            
            # --- ¡NUEVO! Omitir BBOXes ya completados ---
            if bbox_index in completed_bboxes_set:
                print(f"\n--- OMITIENDO BBOX {bbox_index}/{len(bbox_list)} (ya completado) ---")
                continue # Pasa al siguiente BBOX
                
            print(f"\n--- Analizando BBOX {bbox_index}/{len(bbox_list)} ---")

            current_search_url = search_url 
            page_count = 1
            
            count_total_en_bbox = 0
            count_duplicados_en_bbox = 0
            count_nuevas_en_esta_bbox = 0 
            bbox_completado_con_exito = False

            params = {
                "access_token": ACCESS_TOKEN,   
                "fields": "id,thumb_1024_url,geometry",
                "bbox": bbox_str,
                "limit": 5000 
            }

            # --- BUCLE INTERNO: Paginación ---
            try:
                while True:
                    response = requests.get(current_search_url, params=params)
                    response.raise_for_status() 
                    data = response.json()
                    
                    if data and data.get('data'):
                        images_found_on_page = data['data']
                        count_total_en_bbox += len(images_found_on_page)
                        
                        for image_data in images_found_on_page:
                            keys_ok = all(k in image_data for k in ('id', 'thumb_1024_url', 'geometry'))
                            geom_ok = keys_ok and 'coordinates' in image_data.get('geometry', {})

                            if not (keys_ok and geom_ok):
                                continue 

                            image_id = image_data['id']
                            
                            if image_id in ids_globales:
                                count_duplicados_en_bbox += 1
                            else:
                                ids_globales.add(image_id)
                                count_nuevas_en_esta_bbox += 1
                                lon, lat = image_data['geometry']['coordinates']
                                
                                image_info_to_save = {
                                    "id": image_id,
                                    "download_url": image_data['thumb_1024_url'],
                                    "lat": lat,
                                    "lon": lon,
                                    "bbox_index": bbox_index,
                                    "filename": f"bbox_{bbox_index}_{image_id}.jpg"
                                }
                                
                                f_jsonl.write(json.dumps(image_info_to_save) + '\n')
                    
                    # Lógica de Paginación
                    if 'paging' in data and 'next' in data['paging']:
                        current_search_url = data['paging']['next']
                        params = {}
                        page_count += 1
                    else:
                        bbox_completado_con_exito = True # Se llegó al final
                        break 

            except requests.exceptions.RequestException as e:
                print(f"   ERROR en la API para BBOX {bbox_index}: {e}")
                print("   Este BBOX se re-intentará en la próxima ejecución.")
                continue # Pasa al siguiente BBOX
            
            # --- Imprimir el reporte para este BBOX ---
            print(f"   > Total en BBOX: {count_total_en_bbox} imágenes")
            print(f"   > Repetidas (ya vistas): {count_duplicados_en_bbox} imágenes")
            print(f"   > Nuevas añadidas: {count_nuevas_en_esta_bbox} imágenes")
            
            # --- ¡NUEVO! Escribir en el log SÓLO si se completó ---
            if bbox_completado_con_exito:
                f_log.write(f"{bbox_index}\n") # Escribe el número del BBOX
                f_log.flush() # Guarda el cambio en el log inmediatamente
            
            total_duplicados_globales += count_duplicados_en_bbox
            time.sleep(0.5) 

except KeyboardInterrupt:
    print("\n\nProceso interrumpido por el usuario. El progreso ha sido guardado.")
    print("Vuelve a ejecutar el script para continuar desde donde quedaste.")

# --- 4. FASE DE REPORTE FINAL ---
total_imagenes_unicas = len(ids_globales)
print("\n" + "="*30)
print("--- ANÁLISIS COMPLETADO ---")
print(f"Total de BBOXes en el archivo: {len(bbox_list)}")
print(f"BBOXes completados en esta sesión: {len(completed_bboxes_set)}")
print(f"Imágenes Únicas Totales (en archivo): {total_imagenes_unicas}")
print(f"Instancias Duplicadas Totales: {total_duplicados_globales}")
print("="*30)

print(f"\n¡Éxito! Se ha guardado (o actualizado) la información de {total_imagenes_unicas} imágenes en '{OUTPUT_JSONL_FILE}'.")

--- Iniciando Script de REPORTE y GENERACIÓN DE JSON ---
Se analizarán 120 BBOXes.
Cargando IDs de imágenes desde 'descarga_lista.jsonl'...
Se cargaron 68816 IDs únicos ya procesados.

--- Analizando BBOX 1/120 ---
   > Total en BBOX: 4453 imágenes
   > Repetidas (ya vistas): 4453 imágenes
   > Nuevas añadidas: 0 imágenes

--- Analizando BBOX 2/120 ---
   > Total en BBOX: 4830 imágenes
   > Repetidas (ya vistas): 4830 imágenes
   > Nuevas añadidas: 0 imágenes

--- Analizando BBOX 3/120 ---
   > Total en BBOX: 4071 imágenes
   > Repetidas (ya vistas): 4071 imágenes
   > Nuevas añadidas: 0 imágenes

--- Analizando BBOX 4/120 ---
   > Total en BBOX: 4643 imágenes
   > Repetidas (ya vistas): 4643 imágenes
   > Nuevas añadidas: 0 imágenes

--- Analizando BBOX 5/120 ---
   > Total en BBOX: 4466 imágenes
   > Repetidas (ya vistas): 4466 imágenes
   > Nuevas añadidas: 0 imágenes

--- Analizando BBOX 6/120 ---
   > Total en BBOX: 4417 imágenes
   > Repetidas (ya vistas): 4417 imágenes
   > Nuev